In [40]:
import sys
import os
import lxml.etree as ET 
import seaborn as sns

In [94]:
def getDataFromFileRRT(filePath: str) -> tuple:
    tree = ET.parse(filePath)
    root = tree.getroot()
    time = float(root.find('time').text)
    # countOfEdges = int(root.find('countofedges').text)
    distance = float(root.find('distance').text)
    return (time, distance)
    

In [95]:
def getDataFromFileAstar(filePath: str) -> tuple:
    tree = ET.parse(filePath)
    root = tree.getroot()
    log = root.find('log')
    summary = log.find('summary')
    time = float(summary.get('time'))
    
    distance = float(summary.get('length'))
    return (time, distance)



In [100]:
def getAllData(folder: str) -> tuple:
    RRTData = []
    for file in os.listdir(os.path.join(folder, 'RRT')):
        RRTData.append(getDataFromFileRRT(os.path.join(folder, 'RRT', file)))
    AstarData = []
    for file in os.listdir(os.path.join(folder, 'Astar')):
        AstarData.append(getDataFromFileAstar(os.path.join(folder, 'Astar', file)))
    return (RRTData, AstarData)

# Moscow102401

In [106]:
path = 'D:/Projects/PathPlanningAlgorithms-RRT-RRTstar-/tests/analysis/moscow102401'
data = getAllData(path)
print(data[0])
print(data[1])


[(0.121684, 849.500095), (0.058384, 1201.71764), (0.044186, 725.539457), (0.163289, 792.5788), (0.081803, 878.224441), (0.089398, 660.0), (0.155499, 926.134354), (0.044734, 740.0), (0.113823, 807.441223), (0.073173, 730.0)]
[(0.047145, 492.78888), (0.043207, 492.78888), (0.04651, 492.78888), (0.046715, 492.78888), (0.046137, 492.78888), (0.048686, 492.78888), (0.051333, 492.78888), (0.047469, 492.78888), (0.046911, 492.78888), (0.046363, 492.78888)]
